<a href="https://colab.research.google.com/github/abrissard/sappelli_email_classification/blob/main/act1_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U accelerate
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Code/Maitrise/sappelli_data/sappelli_data_act1_encoded.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][0]

{'label': 3,
 'text': 'It was cute!\tGenia FitzGerald\t10/10/2000 03:23 PM\t\t \t\t To: Marie Heard/Enron Communications@Enron Communications\t\t cc: Tana Jones/HOU/ECT@ECT\t\t Subject: Re: HOTDOG !This is sooo adorable!Thanks for inviting me to share Mexican food with you.  Tana said it was YOUR fault I wasn t invited!!!!!Genia----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 03:21 PM -----\t\t----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 02:30 PM ----- - Hotdog PHOTO2.jpg'}

In [ ]:
# prompt: Split the dataset, but keep the HuggingFace format

train_test_dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=False, seed=42)


In [ ]:
# prompt: lenght of train and test

train_length = len(train_test_dataset['train'])
test_length = len(train_test_dataset['test'])

print(f"Train length: {train_length}")
print(f"Test length: {test_length}")


Train length: 456
Test length: 114


In [ ]:
train_test_dataset['test'][0]

{'label': 7,
 'text': 'I will be out tomorrow and I may leave early this afternoon.  Could you email me the deal numbers for the 3 purchases and 12 sales for the month of August?'}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_dataset = train_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Code/Maitrise/sappelli_data/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=145, training_loss=1.2340486723801185, metrics={'train_runtime': 206.4998, 'train_samples_per_second': 11.041, 'train_steps_per_second': 0.702, 'total_flos': 599103257072640.0, 'train_loss': 1.2340486723801185, 'epoch': 5.0})

In [ ]:
# prompt: save model with name act1

model.save_pretrained("/content/drive/MyDrive/Code/Maitrise/sappelli_data/model/act1")


In [ ]:
# prompt: evaluate the results on the test dataset

evaluation = trainer.evaluate(tokenized_dataset["test"])
print(evaluation)


{'eval_loss': 1.1507550477981567, 'eval_runtime': 3.6385, 'eval_samples_per_second': 31.332, 'eval_steps_per_second': 2.199, 'epoch': 5.0}


In [ ]:
# prompt: evaluate using f1 score

from sklearn.metrics import f1_score

# Compute and print the F1 score
f1 = f1_score(tokenized_dataset["test"]["labels"], trainer.predict(tokenized_dataset["test"]).predictions.argmax(axis=-1), average="micro")
print(f"F1 score: {f1}")


KeyError: "Column labels not in the dataset. Current columns in the dataset: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']"